In [1]:
import os
import geopandas as gpd

DATA_PATH = "C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data"
GPKG_FILENAME = "gpw_grassland_fscs.vi.vhr_grid.samples_20000101_20241231_go_epsg.4326_v2.gpkg"

BATCH_SIZE = 10000000

In [2]:
_path = os.path.join(DATA_PATH, GPKG_FILENAME)

processed = 0

while True:
    gdf = gpd.read_file(_path, where=f"rowid >= {processed} AND rowid < {processed + BATCH_SIZE}")

    if gdf.empty:
        print("No more data to process.")
        break

    if gdf.shape[0] + 1 == BATCH_SIZE:
        last_tile_id = gdf.iloc[-1]["tile_id"]
    else:
        last_tile_id = None

    # Save each tile as a separate GeoPackage
    for tile_id in gdf["tile_id"].unique():
        if not "mvp" in tile_id or last_tile_id == tile_id:
            continue 

        tile_gdf = gdf[gdf["tile_id"] == tile_id]
        output_path = os.path.join(DATA_PATH + '/tiles', f"{tile_id}.gpkg")

        os.makedirs(os.path.dirname(output_path), exist_ok=True)

        tile_gdf.to_file(output_path, driver="GPKG")

    processed += len(gdf)

    print(f"Processed up to rowid: {processed}")

Processed up to rowid: 9999999
Processed up to rowid: 19999999
Processed up to rowid: 29999999
Processed up to rowid: 39999999
Processed up to rowid: 49999999
Processed up to rowid: 59999999
Processed up to rowid: 69999999
Processed up to rowid: 79999999
Processed up to rowid: 89999999
Processed up to rowid: 99999999
Processed up to rowid: 102654331
No more data to process.


In [1]:
import geopandas as gpd
import subprocess
from pathlib import Path

# pasta com seus GPKGs
input_dir = Path("C:/Users/tiago/Documents/Workspace/UFG/pfc-cc-2025/data/tiles")
output_dir = input_dir / "rasters"
output_dir.mkdir(exist_ok=True)

# lista todos os arquivos .gpkg
gpkg_files = list(input_dir.glob("*.gpkg"))[:4]

In [3]:
for gpkg in gpkg_files:
    layer = gpkg.stem

    gdf = gpd.read_file(gpkg, layer=layer)
    xmin, ymin, xmax, ymax = gdf.total_bounds

    out_tif = output_dir / f"{layer}.tif"

    cmd = [
        "gdal_rasterize",
        "-a", "google_class",
        "-l", layer,
        "-tr", "10", "10",
        "-te", str(xmin), str(ymin), str(xmax), str(ymax),
        "-ot", "Int16",
        "-a_nodata", "-1",
        str(gpkg),
        str(out_tif)
    ]

    print("Rodando:", " ".join(cmd))
    subprocess.run(cmd, check=True)

Rodando: gdal_rasterize -a google_class -l 1-mvp -tr 10 10 -te 3298060.0 9564778.0 3299080.0 9565798.0 -ot Int16 -a_nodata -1 C:\Users\tiago\Documents\Workspace\UFG\pfc-cc-2025\data\tiles\1-mvp.gpkg C:\Users\tiago\Documents\Workspace\UFG\pfc-cc-2025\data\tiles\rasters\1-mvp.tif


FileNotFoundError: [WinError 2] O sistema não pode encontrar o arquivo especificado

In [ ]:
# Fazer warp dos rasters para o EPSG correto

gdalwarp -t_srs EPSG:32635 -te 624600 7192530 625060 7193000 -tr 10 10 -r mode -dstnodata -1 1-mvp.tif 1-mvp-warped.tif